In [ ]:
import numpy as np
import os, shutil
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img, img_to_array

#Variables to edit---------------
epochs = 1 #Number of epochs
learningrate = 0.0005 #Learning rate for optimizer
imageheight=200 #Image height for model
imagewidth=300  #Image width for model
cutoff=0.7 #Above which is an echolcation and below is abnormal file
mfolder = "./weights1/" #Folder with saved models in h5 format
afolder = "./sorted/predicta/ab/" #Image folder with images to test on model
efolder = "./sorted/ce/e/" #Image folder with images to test on model
#End of variables to edit---------
if not os.path.exists('/models1'): #Create folder for models
    os.makedirs('/models1')
#Tensorboard functions
checkpointvacc = ModelCheckpoint('models1/{epoch:02d}.h5', monitor='val_acc', mode='max',
                                 save_best_only=True, verbose=0)
checkpointvloss = ModelCheckpoint('models1/{epoch:02d}.h5', monitor='val_loss', mode='min',
                                 save_best_only=True, verbose=0)
#Setup to stream images for test and training
train_data = ImageDataGenerator(rescale=1./255)
test_data = ImageDataGenerator(rescale=1./255)
train_generator = train_data.flow_from_directory('sorted/train',batch_size=45,class_mode='binary', target_size=(200,300))
test_generator = test_data.flow_from_directory('sorted/test',batch_size=16,class_mode='binary',target_size=(200,300))
#Create model layers
model = Sequential()
model.add(Conv2D(96, (5, 5), strides=5, input_shape=(200, 300, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(128, (5, 5), strides=5))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))      
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
#Set model parameters
adam = optimizers.adam(lr=learningrate)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
history=model.fit_generator(
        train_generator,
        steps_per_epoch=40,
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=50,
        callbacks=[checkpointvloss,checkpointvacc])

#Evaluate model---------------------
for mdl in range(16): #Remove unwanted models
    if os.path.exists(mfolder+str(mdl)+".h5"):
        os.remove(mfolder+str(mdl)+".h5")
    if os.path.exists(mfolder+"0"+str(mdl)+".h5"):
        os.remove(mfolder+"0"+str(mdl)+".h5")
shutil.copy2('/models/CNN200x300b.h5', '/models1')       
mfilelist = [f for f in os.listdir(mfolder) if os.path.isfile(os.path.join(mfolder, f))] #Make list of model filenames
afilelist = [f for f in os.listdir(afolder) if os.path.isfile(os.path.join(afolder, f))] #Make list of image filenames
efilelist = [f for f in os.listdir(efolder) if os.path.isfile(os.path.join(efolder, f))] #Make list of image filenames
abcount=echcount=abest=ambest=0
abmodel = [0] * len(mfilelist)
for mdl in range(len(mfilelist)):
    model = load_model(mfolder+mfilelist[mdl])
    for inum in range(len(afilelist)):#Go through images 1 by 1 and run predictions on model
        image = load_img(afolder + afilelist[inum], target_size=(imageheight,imagewidth))
        imageresize = image.resize((imagewidth, imageheight)) #Resize image for desired CNN model
        imagearray = img_to_array(imageresize) #Make image into an array
        imagearray = np.expand_dims(imagearray, 0) #Add 4th dimension to array for CNN -> will get error otherwise
        imagearray = imagearray.astype('float32')/255 #Convert array so that its predictions are readable
        prediction=model.predict(imagearray) #Run prediction on image-Returns list of float predictions
        if prediction[0]<cutoff:
            abcount=abcount+1
    abmodel[mdl]= (abcount*100)/len(afilelist)
    if ((abcount*100)/len(afilelist))>=abest:
        abest = (abcount*100)/len(afilelist)
        ambest = mdl
   
        

